- Tensorflow 2.2.0
- Python 3.6.13
- opencv 4.5.4.60

In [11]:
import os
import cv2
import json
import numpy as np
from tensorflow.keras.models import load_model

Link para download do facenet.h5: https://github.com/nyoki-mtl/keras-facenet

In [ ]:
embedder_path = 'facenet_keras.h5'
embedder = load_model(embedder_path)

In [15]:
def load_and_preprocess_face(image_path):
    image = cv2.imread(image_path)

    if image is None:
        print(f"Erro ao carregar imagem: {image_path}")
        return None  # Retorna None para ser tratado depois

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV carrega em BGR, convertendo para RGB
    image = cv2.resize(image, (160, 160))  # facenet exige imagens 160x160
    image = np.array(image, dtype='float32') / 255.0  # Normalizar para [0,1]
    return image

In [16]:
# Gerar embeddings e salvar
def generate_and_save_embeddings(root_directory, output_file):
    embeddings = {}

    for subdir, _, files in os.walk(root_directory):
        label = os.path.basename(subdir)  # Nome do subdiretório (ex: nome da pessoa)
        embeddings[label] = []

        for filename in files:
            image_path = os.path.join(subdir, filename)
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  
                face = load_and_preprocess_face(image_path)
                face = np.expand_dims(face, axis=0)  # Adicionar dimensão batch
                embedding = embedder.predict(face)[0].tolist()  # Converter para lista
                embeddings[label].append(embedding)  # Salvar embedding

    # Salvar embeddings em um arquivo JSON
    with open(output_file, 'w') as f:
        json.dump(embeddings, f)

In [ ]:
# Diretório das faces e arquivo de saída
faces_directory = "Reconhecimento Facial\\faces"
output_file = "embeddings.json"

In [18]:
# Gerar e salvar embeddings
generate_and_save_embeddings(faces_directory, output_file)
print(f"Embeddings salvos em {output_file}")

Embeddings salvos em embeddings.json
